# Cryptohopper Kucoin

## ✳️ Required User Inputs

In [1]:
# KUCOIN
source_filename = "cryptohopper_kucoin_2021_03_21.csv"
initial_investment = 8297

## Setting up the stage!

In [2]:
import pandas as pd
import numpy as np
import datetime as dt

In [3]:
folder = "cryptohopper/data/"
source = folder + source_filename
data = pd.read_csv(source, index_col="Id", header=0, parse_dates=["Date"])

## Data Cleanse

In [4]:
# Convert "Result" feature from String to Float
data["Result"] = data["Result"].str.rstrip('%').astype('float').div(100)

# Clean up Trigger/Signals Names
data["Trigger"] = data["Trigger"].str.lstrip("Signal From")

## Add Features

In [5]:
# Add "BuyTrigger" to "sell" rows
data["BuyTrigger"] = data["Buy order ID"].apply(lambda x: data.loc[x,"Trigger"] \
                                                if x != 0 else np.NaN)

# Remove AUTO SYNCs
data = data[(data["Trigger"] != "Auto Synced") & 
            (data["BuyTrigger"] != "Auto Synced")]

# Add "BuyValue" to "sell" rows
data["BuyValue"] = data["Buy order ID"].apply(lambda x: data.loc[x,"Order Value"] \
                                                if x != 0 else np.NaN)

# Add "BuyFee" to "sell" rows
data["BuyFee"] = data["Buy order ID"].apply(lambda x: data.loc[x,"Fee"] \
                                                if x != 0 else np.NaN)

# Add "Result USDT" to "sell" rows
# WITH FEES
data["Result USDT"] = (data["BuyValue"] * data["Result"]) - (data["Fee"] + data["BuyFee"])
# WITHOUT FEES
# data["Result USDT"] = (data["BuyValue"] * data["Result"])

## Profit Total

In [6]:
df = data
df["Result USDT"].sum()

151.18363808210427

## Profit Per Month

In [7]:
df = data
df = df.groupby([df["Date"].dt.strftime('%Y %B')])["Result USDT"].sum()
df = pd.DataFrame(df)
df["Investment"] = np.NaN
df["Investment"][0] = initial_investment
df["Return %"] = df["Result USDT"] * 100 / df["Investment"]
df

,Result USDT,Investment,Return %
Date,,,
2021 March,151.183638,8297.0,1.822148


## Profit Per Day

In [8]:
df = data
df = df.groupby([df["Date"].dt.date])["Result USDT"].sum()
df

Date
2021-03-05     22.341295
2021-03-06     35.932273
2021-03-07     35.805051
2021-03-08     40.138475
2021-03-09     50.895131
2021-03-10    259.537075
2021-03-11    -78.009311
2021-03-12    -96.000673
2021-03-13    278.369928
2021-03-14      9.311981
2021-03-15   -208.228616
2021-03-16    -89.129914
2021-03-17     23.073713
2021-03-18    -18.119676
2021-03-19     90.879164
2021-03-20     29.878225
2021-03-21   -235.490482
Name: Result USDT, dtype: float64

## Results and Number of Closed Trades per Signaler / Per Month

In [9]:
df = data
df = df.groupby([df["Date"].dt.strftime('%Y %B'), "BuyTrigger"])
df = df.agg({"Result USDT": "sum", "Result" : "count"})
df.rename({"Result": "Closed"}, inplace=True, axis=1)
df

Result USDT  Closed
Date       BuyTrigger                                           
2021 March Altcoin Ninjas                     -73.479762      87
           Crypto Quality Signals              18.399743      20
           Jackrabbit Premium                  28.702680      15
           Killer Whale Bank Robber Signals    88.716460     258
           Oxford Crypto Signals               84.409330     190
           Profit Scalping Signals   DEX     -216.281861     688
           trategy Nakamoto   Signals FREE    220.717048     337

## Number of BUY Signals per Signaler

In [10]:
df = data
df = df[df["Type"] == "buy"]
df.groupby("Trigger")["Trigger"].count()

Trigger
Altcoin Ninjas                       99
Crypto Quality Signals               22
Jackrabbit Premium                   17
Killer Whale Bank Robber Signals    271
Oxford Crypto Signals               210
Profit Scalping Signals   DEX       744
trategy Nakamoto   Signals FREE     349
Name: Trigger, dtype: int64

## Number of BUY Transactions per Date

In [11]:
df = data
df = df[df["Type"] == "buy"]
df.groupby([df["Date"].dt.date])["Trigger"].count()

Date
2021-03-05     37
2021-03-06     42
2021-03-07     47
2021-03-08     35
2021-03-09     69
2021-03-10    240
2021-03-11     71
2021-03-12     95
2021-03-13    185
2021-03-14     56
2021-03-15    141
2021-03-16    194
2021-03-17    133
2021-03-18    105
2021-03-19    103
2021-03-20    117
2021-03-21     42
Name: Trigger, dtype: int64

## Number of BUY Transactions per Date / Signaler

In [12]:
df = data
df = df[df["Type"] == "buy"]
df.groupby([df["Date"].dt.date, "Trigger"])["Trigger"].count().tail(15)

Date        Trigger                         
2021-03-19  Killer Whale Bank Robber Signals    15
            Oxford Crypto Signals               12
            Profit Scalping Signals   DEX       55
            trategy Nakamoto   Signals FREE      9
2021-03-20  Altcoin Ninjas                       4
            Jackrabbit Premium                   2
            Killer Whale Bank Robber Signals    22
            Oxford Crypto Signals               23
            Profit Scalping Signals   DEX       57
            trategy Nakamoto   Signals FREE      9
2021-03-21  Altcoin Ninjas                       6
            Killer Whale Bank Robber Signals     9
            Oxford Crypto Signals                6
            Profit Scalping Signals   DEX       17
            trategy Nakamoto   Signals FREE      4
Name: Trigger, dtype: int64

## Assertiviness Per Signaler

In [13]:
df = data
df = df.groupby("BuyTrigger")["Trigger"].value_counts(normalize=True) * 100
df

BuyTrigger                        Trigger                         
Altcoin Ninjas                    Trailing Stop Loss                   84.090909
                                  top Loss                             14.772727
                                  Killer Whale Bank Robber Signals      1.136364
Crypto Quality Signals            Trailing Stop Loss                   95.238095
                                  top Loss                              4.761905
Jackrabbit Premium                Trailing Stop Loss                  100.000000
Killer Whale Bank Robber Signals  Trailing Stop Loss                   90.310078
                                  top Loss                              8.527132
                                  Killer Whale Bank Robber Signals      0.775194
                                  Oxford Crypto Signals                 0.387597
Oxford Crypto Signals             Trailing Stop Loss                   87.958115
                                  top Loss

## Negative Results from triggers other than Stop Loss

In [14]:
df = data
mask = (df["Result"] < 0) & (df["Trigger"] != "top Loss")
df = df[mask]
len(df)

165

In [15]:
df["Result USDT"].sum()

-323.74120704947836

## Stop Losses

In [16]:
df = data
len(df[df["Trigger"] == "top Loss"])

152

## Save Clean Data

In [17]:
destination_filename = source_filename.split(".")[0] + "_clean.csv"
destination = folder + destination_filename
data.to_csv(destination, index = True) 